In [1]:
import http.server
import socketserver
import json
import time
import os.path
import Crypto
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES
from Crypto import Random
import secrets
import base64
import hashlib

In [2]:
current_time_millis = lambda: int(round(time.time() * 1000))

In [3]:
path_public_rsa = 'public_rsa'
path_get_file = 'get_file'
path_session_key = 'session_key'
path_login = 'login'
key_public_exponent = 'exponent'
key_modulus = 'modulus'
key_filename = 'file'
key_username = 'username'
key_password = 'password'
key_error = 'error'
key_content = 'content'
key_iv = 'iv'
key_extra_symbols = 'extra_symbols'
session_key_expiration_period = 1 * 60 * 1000 # 1 minute
public_key_filename = 'public_key.txt'

class SessionKey:
    
    def __init__(self, key):
        self.key = key
        self.created = current_time_millis()
    
    def is_expired(self):
        return current_time_millis() - self.created > session_key_expiration_period
    
class DataHolder:
    
    def __init__(self):
        self.session_key = None
        self.public_exponent = None
        self.modulus = None
        if os.path.isfile(public_key_filename): 
            file = open("public_key.txt", "r")
            content = file.read()
            if (content is not None) and (len(content) > 0):
                try:
                    json_public_key = json.loads(content)
                    self.public_exponent = int(json_public_key[key_public_exponent])
                    self.modulus = int(json_public_key[key_modulus])
                except Exception as e:
                    print(e)
        print('initial exponent: ', self.public_exponent, ', modulus:', self.modulus)
        
    def check_username_and_password(self, username, password_json):
        if not os.path.isfile(username + '.txt'):
            return 'WRONG_USERNAME'
        if self.session_key is None:
            return 'NO_SESSION_KEY'
        try:
        iv = base64.b64decode(bytes(password_json[key_iv], 'utf-8'))
        encrypted_password = base64.b64decode(bytes(password_json[key_content], 'utf-8'))
        aes = AES.new(self.session_key.key, AES.MODE_OFB, iv)
        extra_symbols = password_json[key_extra_symbols]
        decrypted_password = aes.decrypt(encrypted_password)[:-extra_symbols]
        decrypted_password_hash = hashlib.md5(decrypted_password).hexdigest()
        file = open(username + '.txt', 'r')
        actual_password_hash = file.read()
        file.close()
        if decrypted_password_hash != actual_password_hash:
            return 'WRONG_PASSWORD'
        else:
            return 'OK'
        except Exception as e:
            print(e)
        return 'ERROR'

    def get_string_from_bytes(self, bytes_arr):
        return str(base64.b64encode(bytes_arr), "utf-8")
    
    def get_file_response(self, filename):
        response = {}
        if self.session_key is None or self.session_key.is_expired():
            response[key_error] = 'session_key_expired'
        elif not os.path.isfile(filename):
            response[key_error] = 'no_file'
        else:
            iv = Random.new().read(16)
            aes = AES.new(self.session_key.key, AES.MODE_OFB, iv)
            
            file = open(filename, "r")
            file_content = file.read()
            file.close()
                
            extra_symbols = 16 - len(file_content) % 16
            extra_data = ' ' * extra_symbols
            file_content += extra_data
            response[key_content] = self.get_string_from_bytes(aes.encrypt(file_content))
            response[key_iv] = self.get_string_from_bytes(iv)
            response[key_extra_symbols] = extra_symbols
        return json.dumps(response)

In [4]:
data_holder = DataHolder()

class MyHTTPRequestHandler(http.server.BaseHTTPRequestHandler):

    def do_GET(self):
        split_by_question = self.path.split('?')
        path = split_by_question[0]
        argument_dict = {}
        if len(split_by_question) > 1:
            argument_pairs = split_by_question[1].split('&')
            for pair in argument_pairs:
                key_and_value = pair.split('=')
                argument_dict[key_and_value[0]] = key_and_value[1]
        if (path[0] == '/'):
            path = path[1:]
        if path[-1] == '/':
            path = path[:-1]
        if path == path_get_file:
            self.handle_get_file(argument_dict)
        elif path == path_session_key:
            self.handle_session_key()
        else:
            self.send_response(404)
            self.end_headers()            

    def do_POST(self):
        split_by_question = self.path.split('?')
        path = split_by_question[0]
        if (path[0] == '/'):
            path = path[1:]
        if path[-1] == '/':
            path = path[:-1]
        content_len = int(self.headers.get('Content-Length'))
        post_body = self.rfile.read(content_len)
        if path == path_login:
            self.handle_login(post_body)
        elif path == path_public_rsa:
            self.handle_public_rsa(post_body)
        else:
            self.send_response(404)
            self.end_headers()            

    def handle_login(self, post_body):
        print('received post:', post_body)
        try:
            post_json = json.loads(post_body)
            username = post_json[key_username]
            password_json = json.loads(post_json[key_password])
            result = data_holder.check_username_and_password(username, password_json)
            print('username:', username)
            self.send_response(200)
            self.end_headers()            
            self.wfile.write(bytearray(result, 'utf8'))
        except Exception as e:
            print(e)
            self.send_response(413)
            self.end_headers()                        

    def handle_public_rsa(self, post_body):
        try:
            post_json = json.loads(post_body)
            data_holder.public_exponent = int(post_json[key_public_exponent])
            data_holder.modulus = int(post_json[key_modulus])
            json_public_key = {}
            json_public_key[key_public_exponent] = data_holder.public_exponent
            json_public_key[key_modulus] = data_holder.modulus
            file = open(public_key_filename, "w+")
            file.write(json.dumps(json_public_key))
            file.close()
            print('handle_public_rsa: exp:', data_holder.public_exponent, ', modulus:', data_holder.modulus)
            self.send_response(200)
            self.end_headers()     
        except Exception as e:
            print(e)
            self.send_response(413)
            self.end_headers()     

    def handle_get_file(self, arguments):
        try:
            filename = arguments[key_filename]
            print('handle_get_file: ', filename)
            response = data_holder.get_file_response(filename)
            self.send_response(200)
            self.end_headers()    
            self.wfile.write(bytes(response, 'utf-8'))
        except Exception as e:
            print(e)
            self.send_response(413)
            self.end_headers()    

    def handle_session_key(self):
        print('handle_session_key')
        try:
            session_key = secrets.token_bytes(32)
            data_holder.session_key = SessionKey(session_key)
            public_key = RSA.construct((data_holder.modulus, data_holder.public_exponent))
            encrypted_session_key = public_key.encrypt(session_key, None)

            self.send_response(200)
            self.end_headers()    
            self.wfile.write(encrypted_session_key[0])
        except Exception as e:
            print(e)
            self.send_response(413)
            self.end_headers()                    

initial exponent:  65537 , modulus: 72822657943202290206626939526234807206638650338287413590814555771313692244033


In [ ]:
port = 8000
Handler = http.server.SimpleHTTPRequestHandler

httpd = http.server.HTTPServer(('localhost', port), MyHTTPRequestHandler)
print("serving at port", port)
httpd.serve_forever()

serving at port 8000
handle_session_key


127.0.0.1 - - [23/Oct/2019 23:49:36] "GET /session_key HTTP/1.1" 200 -


received post: b'{"username":"dzina","password":"{\\"content\\":\\"56chJXkeJ0c21bh8MwXX3Q==\\\\n\\",\\"iv\\":\\"gGYFRaFYpCOu3G2QnL96fg==\\\\n\\",\\"extra_symbols\\":9}"}'
encrypted_password: b"\xe7\xa7!%y\x1e'G6\xd5\xb8|3\x05\xd7\xdd" iv: b'\x80f\x05E\xa1X\xa4#\xae\xdcm\x90\x9c\xbfz~'
decrypted_password: b'TM\x1a\x954\x18i'
username: dzina


127.0.0.1 - - [23/Oct/2019 23:49:48] "POST /login HTTP/1.1" 200 -


handle_public_rsa: exp: 65537 , modulus: 92707137204447428270081704947984099451030584690215633445543209833146268241119


...

127.0.0.1 - - [24/Oct/2019 00:11:35] "POST /login HTTP/1.1" 200 -


received post: b'{"username":"sweta","password":"{\\"content\\":\\"YHDgEM4+7F5Bj4GyFL71zA==\\\\n\\",\\"iv\\":\\"VSKTSy2U69pl3jcRipgrHg==\\\\n\\",\\"extra_symbols\\":10}"}'
encrypted_password: b'`p\xe0\x10\xce>\xec^A\x8f\x81\xb2\x14\xbe\xf5\xcc' iv: b'U"\x93K-\x94\xeb\xdae\xde7\x11\x8a\x98+\x1e'
decrypted_password: b'sqeeta'
username: sweta


127.0.0.1 - - [24/Oct/2019 00:11:44] "POST /login HTTP/1.1" 200 -


received post: b'{"username":"sweta","password":"{\\"content\\":\\"Mu7JSVKTA\\\\\\/z6DI+f4CcscA==\\\\n\\",\\"iv\\":\\"YRQxTpYAMpv06bMO7\\\\\\/Me9Q==\\\\n\\",\\"extra_symbols\\":10}"}'
encrypted_password: b"2\xee\xc9IR\x93\x03\xfc\xfa\x0c\x8f\x9f\xe0',p" iv: b'a\x141N\x96\x002\x9b\xf4\xe9\xb3\x0e\xef\xf3\x1e\xf5'
decrypted_password: b'ssweta'
username: sweta


127.0.0.1 - - [24/Oct/2019 00:11:49] "POST /login HTTP/1.1" 200 -
